In [1]:
import os

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/husein/t5/prepare/mesolitica-tpu.json'

In [2]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '3'

from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_problems
from tensor2tensor.data_generators import translate
from tensor2tensor.utils import registry
from tensor2tensor import problems
import tensorflow as tf
import os
import logging

logger = logging.getLogger()
tf.logging.set_verbosity(tf.logging.DEBUG)

In [3]:
import sentencepiece as spm

vocab = 'sp10m.cased.t5.model'
sp = spm.SentencePieceProcessor()
sp.Load(vocab)

class Encoder:
    def __init__(self, sp):
        self.sp = sp
        self.vocab_size = sp.GetPieceSize() + 100
    
    def encode(self, s):
        return self.sp.EncodeAsIds(s)
    
    def decode(self, ids, strip_extraneous=False):
        return self.sp.DecodeIds(list(ids))
    
encoder = Encoder(sp)

In [4]:
from tqdm import tqdm
from glob import glob

@registry.register_problem
class Seq2Seq(text_problems.Text2TextProblem):

    @property
    def approx_vocab_size(self):
        return 32100
    
    @property
    def is_generate_per_split(self):
        return False
            
    def feature_encoders(self, data_dir):
        encoder = Encoder(sp)
        return {
            "inputs": encoder,
            "targets": encoder
        }


In [5]:
DATA_DIR = os.path.expanduser('t2t-summarization/data')
TMP_DIR = os.path.expanduser('t2t-summarization/tmp')

In [6]:
PROBLEM = 'seq2_seq'
t2t_problem = problems.problem(PROBLEM)

In [7]:
import tensorflow as tf
import os

ckpt_path = tf.train.latest_checkpoint('gs://mesolitica-tpu-general/t2t-summarization-small')
ckpt_path

'gs://mesolitica-tpu-general/t2t-summarization-small/model.ckpt-50000'

In [8]:
from tensor2tensor import models
from tensor2tensor import problems
from tensor2tensor.layers import common_layers
from tensor2tensor.utils import trainer_lib
from tensor2tensor.utils import t2t_model
from tensor2tensor.utils import registry
from tensor2tensor.utils import metrics
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_problems
from tensor2tensor.data_generators import translate
from tensor2tensor.utils import registry

In [9]:
from tensor2tensor.layers import modalities
from tensor2tensor.layers import common_layers

def top_p_logits(logits, p):
    with tf.variable_scope('top_p_logits'):
        logits_sort = tf.sort(logits, direction = 'DESCENDING')
        probs_sort = tf.nn.softmax(logits_sort)
        probs_sums = tf.cumsum(probs_sort, axis = 1, exclusive = True)
        logits_masked = tf.where(
            probs_sums < p, logits_sort, tf.ones_like(logits_sort) * 1000
        )  # [batchsize, vocab]
        min_logits = tf.reduce_min(
            logits_masked, axis = 1, keepdims = True
        )  # [batchsize, 1]
        return tf.where(
            logits < min_logits,
            tf.ones_like(logits, dtype = logits.dtype) * -1e10,
            logits,
        )


def sample(translate_model, features):
    logits, losses = translate_model(features)
    logits_shape = common_layers.shape_list(logits)
    logits_p = logits[:,0,:,0,:] / translate_model.hparams.sampling_temp
    logits_p = top_p_logits(logits_p, translate_model.hparams.top_p)
    reshaped_logits = tf.reshape(logits_p, [-1, logits_shape[-1]])
    choices = tf.multinomial(reshaped_logits, 1)
    samples = tf.reshape(choices, logits_shape[:-1])
    return samples, logits, losses

def nucleus_sampling(translate_model, features, decode_length):
    """A slow greedy inference method.
    Quadratic time in decode_length.
    Args:
      features: an map of string to `Tensor`
      decode_length: an integer.  How many additional timesteps to decode.
    Returns:
      A dict of decoding results {
          "outputs": integer `Tensor` of decoded ids of shape
              [batch_size, <= decode_length] if beam_size == 1 or
              [batch_size, top_beams, <= decode_length]
          "scores": None
          "logits": `Tensor` of shape [batch_size, time, 1, 1, vocab_size].
          "losses": a dictionary: {loss-name (string): floating point `Scalar`}
      }
    """
    if not features:
        features = {}
    inputs_old = None
    if 'inputs' in features and len(features['inputs'].shape) < 4:
        inputs_old = features['inputs']
        features['inputs'] = tf.expand_dims(features['inputs'], 2)
    # Save the targets in a var and reassign it after the tf.while loop to avoid
    # having targets being in a 'while' frame. This ensures targets when used
    # in metric functions stays in the same frame as other vars.
    targets_old = features.get('targets', None)

    target_modality = translate_model._problem_hparams.modality['targets']

    def infer_step(recent_output, recent_logits, unused_loss):
        """Inference step."""
        if not tf.executing_eagerly():
            if translate_model._target_modality_is_real:
                dim = translate_model._problem_hparams.vocab_size['targets']
                if dim is not None and hasattr(
                    translate_model._hparams, 'vocab_divisor'
                ):
                    dim += (-dim) % translate_model._hparams.vocab_divisor
                recent_output.set_shape([None, None, None, dim])
            else:
                recent_output.set_shape([None, None, None, 1])
        padded = tf.pad(recent_output, [[0, 0], [0, 1], [0, 0], [0, 0]])
        features['targets'] = padded
        # This is inefficient in that it generates samples at all timesteps,
        # not just the last one, except if target_modality is pointwise.
        samples, logits, losses = sample(translate_model, features)
        # Concatenate the already-generated recent_output with last timestep
        # of the newly-generated samples.
        top = translate_model._hparams.top.get(
            'targets', modalities.get_top(target_modality)
        )
        if getattr(top, 'pointwise', False):
            cur_sample = samples[:, -1, :, :]
        else:
            cur_sample = samples[
                :, common_layers.shape_list(recent_output)[1], :, :
            ]
        if translate_model._target_modality_is_real:
            cur_sample = tf.expand_dims(cur_sample, axis = 1)
            samples = tf.concat([recent_output, cur_sample], axis = 1)
        else:
            cur_sample = tf.to_int64(tf.expand_dims(cur_sample, axis = 1))
            samples = tf.concat([recent_output, cur_sample], axis = 1)
            if not tf.executing_eagerly():
                samples.set_shape([None, None, None, 1])

        # Assuming we have one shard for logits.
        logits = tf.concat([recent_logits, logits[:, -1:]], 1)
        loss = sum([l for l in losses.values() if l is not None])
        return samples, logits, loss

    # Create an initial output tensor. This will be passed
    # to the infer_step, which adds one timestep at every iteration.
    if 'partial_targets' in features:
        initial_output = tf.to_int64(features['partial_targets'])
        while len(initial_output.get_shape().as_list()) < 4:
            initial_output = tf.expand_dims(initial_output, 2)
        batch_size = common_layers.shape_list(initial_output)[0]
    else:
        batch_size = common_layers.shape_list(features['inputs'])[0]
        if translate_model._target_modality_is_real:
            dim = translate_model._problem_hparams.vocab_size['targets']
            if dim is not None and hasattr(
                translate_model._hparams, 'vocab_divisor'
            ):
                dim += (-dim) % translate_model._hparams.vocab_divisor
            initial_output = tf.zeros(
                (batch_size, 0, 1, dim), dtype = tf.float32
            )
        else:
            initial_output = tf.zeros((batch_size, 0, 1, 1), dtype = tf.int64)
    # Hack: foldl complains when the output shape is less specified than the
    # input shape, so we confuse it about the input shape.
    initial_output = tf.slice(
        initial_output, [0, 0, 0, 0], common_layers.shape_list(initial_output)
    )
    target_modality = translate_model._problem_hparams.modality['targets']
    if (
        target_modality == modalities.ModalityType.CLASS_LABEL
        or translate_model._problem_hparams.get('regression_targets')
    ):
        decode_length = 1
    else:
        if 'partial_targets' in features:
            prefix_length = common_layers.shape_list(
                features['partial_targets']
            )[1]
        else:
            prefix_length = common_layers.shape_list(features['inputs'])[1]
        decode_length = prefix_length + decode_length

    # Initial values of result, logits and loss.
    result = initial_output
    vocab_size = translate_model._problem_hparams.vocab_size['targets']
    if vocab_size is not None and hasattr(
        translate_model._hparams, 'vocab_divisor'
    ):
        vocab_size += (-vocab_size) % translate_model._hparams.vocab_divisor
    if translate_model._target_modality_is_real:
        logits = tf.zeros((batch_size, 0, 1, vocab_size))
        logits_shape_inv = [None, None, None, None]
    else:
        # tensor of shape [batch_size, time, 1, 1, vocab_size]
        logits = tf.zeros((batch_size, 0, 1, 1, vocab_size))
        logits_shape_inv = [None, None, None, None, None]
    if not tf.executing_eagerly():
        logits.set_shape(logits_shape_inv)

    loss = 0.0

    def while_exit_cond(
        result, logits, loss
    ):  # pylint: disable=unused-argument
        """Exit the loop either if reach decode_length or EOS."""
        length = common_layers.shape_list(result)[1]

        not_overflow = length < decode_length

        if translate_model._problem_hparams.stop_at_eos:

            def fn_not_eos():
                return tf.not_equal(  # Check if the last predicted element is a EOS
                    tf.squeeze(result[:, -1, :, :]), 1
                )

            not_eos = tf.cond(
                # We only check for early stopping if there is at least 1 element (
                # otherwise not_eos will crash).
                tf.not_equal(length, 0),
                fn_not_eos,
                lambda: True,
            )

            return tf.cond(
                tf.equal(batch_size, 1),
                # If batch_size == 1, we check EOS for early stopping.
                lambda: tf.logical_and(not_overflow, not_eos),
                # Else, just wait for max length
                lambda: not_overflow,
            )
        return not_overflow

    result, logits, loss = tf.while_loop(
        while_exit_cond,
        infer_step,
        [result, logits, loss],
        shape_invariants = [
            tf.TensorShape([None, None, None, None]),
            tf.TensorShape(logits_shape_inv),
            tf.TensorShape([]),
        ],
        back_prop = False,
        parallel_iterations = 1,
    )
    if inputs_old is not None:  # Restore to not confuse Estimator.
        features['inputs'] = inputs_old
    # Reassign targets back to the previous value.
    if targets_old is not None:
        features['targets'] = targets_old
    losses = {'training': loss}
    if 'partial_targets' in features:
        partial_target_length = common_layers.shape_list(
            features['partial_targets']
        )[1]
        result = tf.slice(
            result, [0, partial_target_length, 0, 0], [-1, -1, -1, -1]
        )
    return {
        'outputs': result,
        'scores': None,
        'logits': logits,
        'losses': losses,
    }

In [10]:
class Model:
    def __init__(self, HPARAMS = "transformer_base", DATA_DIR = 't2t/data'):
        
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None, None])
        self.top_p = tf.placeholder(tf.float32, None)
        
        self.X_seq_len = tf.count_nonzero(self.X, 1, dtype=tf.int32)
        self.maxlen_decode = tf.reduce_max(self.X_seq_len)
        #self.maxlen_decode = tf.placeholder(tf.int32, None)
        
        x = tf.expand_dims(tf.expand_dims(self.X, -1), -1)
        y = tf.expand_dims(tf.expand_dims(self.Y, -1), -1)
        
        features = {
            "inputs": x,
            "targets": y,
            "target_space_id": tf.constant(1, dtype=tf.int32),
        }
        self.features = features
        
        Modes = tf.estimator.ModeKeys
        hparams = trainer_lib.create_hparams(HPARAMS, data_dir=DATA_DIR, problem_name=PROBLEM)
        hparams.filter_size = 2048
        hparams.hidden_size = 512
        hparams.num_heads = 8
        hparams.num_hidden_layers = 6
        hparams.vocab_divisor = 128
        hparams.dropout = 0.1
        hparams.max_length = 1024
        
        hparams.label_smoothing = 0.0
        hparams.shared_embedding_and_softmax_weights = False
        hparams.eval_drop_long_sequences = True
        hparams.max_length = 1024
        hparams.multiproblem_mixing_schedule = 'pretrain'
        
        hparams.symbol_modality_num_shards = 1
        hparams.attention_dropout_broadcast_dims = '0,1'
        hparams.relu_dropout_broadcast_dims = '1'
        hparams.layer_prepostprocess_dropout_broadcast_dims = '1'
        
        translate_model = registry.model('transformer')(hparams, Modes.PREDICT)
        self.translate_model = translate_model
        logits, _ = translate_model(features)
        self.logits = logits
        translate_model.hparams.top_p = self.top_p
        
        with tf.variable_scope(tf.get_variable_scope(), reuse=True):
            self.fast_result = translate_model._greedy_infer(features, self.maxlen_decode)["outputs"]
            self.beam_result = translate_model._beam_decode_slow(
                features, self.maxlen_decode, beam_size=3, 
                top_beams=1, alpha=0.5)["outputs"]
            self.nucleus_result = nucleus_sampling(translate_model, features, self.maxlen_decode)["outputs"]
            self.nucleus_result = self.nucleus_result[:,:,0,0]
        
        self.fast_result = tf.identity(self.fast_result, name = 'greedy')
        self.beam_result = tf.identity(self.beam_result, name = 'beam')
        self.nucleus_result = tf.identity(self.nucleus_result, name = 'nucleus')
        
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Model()

Instructions for updating:
reduction_indices is deprecated, use axis instead


Instructions for updating:
reduction_indices is deprecated, use axis instead


INFO:tensorflow:Setting T2TModel mode to 'infer'


INFO:tensorflow:Setting T2TModel mode to 'infer'


INFO:tensorflow:Setting hparams.dropout to 0.0


...

INFO:tensorflow:Building model body


INFO:tensorflow:Transforming body output with symbol_modality_32128_512.top


INFO:tensorflow:Transforming body output with symbol_modality_32128_512.top


Instructions for updating:
Use `tf.random.categorical` instead.


Instructions for updating:
Use `tf.random.categorical` instead.


In [11]:
tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)

[<tf.Variable 'transformer/symbol_modality_32128_512/input_emb/weights_0:0' shape=(32128, 512) dtype=float32>,
 <tf.Variable 'transformer/symbol_modality_32128_512/target_emb/weights_0:0' shape=(32128, 512) dtype=float32>,
 <tf.Variable 'transformer/body/target_space_embedding/kernel:0' shape=(32, 512) dtype=float32>,
 <tf.Variable 'transformer/body/encoder/layer_0/self_attention/layer_prepostprocess/layer_norm/layer_norm_scale:0' shape=(512,) dtype=float32>,
 <tf.Variable 'transformer/body/encoder/layer_0/self_attention/layer_prepostprocess/layer_norm/layer_norm_bias:0' shape=(512,) dtype=float32>,
 <tf.Variable 'transformer/body/encoder/layer_0/self_attention/multihead_attention/q/kernel:0' shape=(512, 512) dtype=float32>,
 <tf.Variable 'transformer/body/encoder/layer_0/self_attention/multihead_attention/k/kernel:0' shape=(512, 512) dtype=float32>,
 <tf.Variable 'transformer/body/encoder/layer_0/self_attention/multihead_attention/v/kernel:0' shape=(512, 512) dtype=float32>,
 <tf.Vari

In [12]:
var_lists = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
saver = tf.train.Saver(var_list = var_lists)
saver.restore(sess, ckpt_path)

INFO:tensorflow:Restoring parameters from gs://mesolitica-tpu-general/t2t-summarization-small/model.ckpt-50000


INFO:tensorflow:Restoring parameters from gs://mesolitica-tpu-general/t2t-summarization-small/model.ckpt-50000


In [13]:
import re
from unidecode import unidecode

def cleaning(string):
    return re.sub(r'[ ]+', ' ', unidecode(string.replace('\n', ' '))).strip()

In [14]:
import re
from unidecode import unidecode
from malaya.text.rules import normalized_chars

def filter_news(string):
    string = string.lower()
    return 'javascript is disabled' in string or 'requires javascript' in string or 'javascript' in string \
    or 'président' in string

def make_cleaning(s, c_dict):
    s = s.translate(c_dict)
    return s

def transformer_textcleaning(string):
    """
    use by any transformer model before tokenization
    """
    string = unidecode(string)
    string = ' '.join(
        [make_cleaning(w, normalized_chars) for w in string.split()]
    )
    string = re.sub('\(dot\)', '.', string)
    string = (
        re.sub(re.findall(r'\<a(.*?)\>', string)[0], '', string)
        if (len(re.findall(r'\<a (.*?)\>', string)) > 0)
        and ('href' in re.findall(r'\<a (.*?)\>', string)[0])
        else string
    )
    string = re.sub(
        r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', ' ', string
    )
    string = string.replace('\n', ' ')
    string = re.sub(r'[ ]+', ' ', string).strip().split()
    string = [w for w in string if w[0] != '@']
    return ' '.join(string)

In [15]:
string = """
KUALA LUMPUR: Hakim Mahkamah Tinggi, Mohd Nazlan Mohd Ghazali menyifatkan kes penyelewengan dana RM42 juta milik SRC International Sdn Bhd dihadapi Datuk Seri Najib Razak adalah kesalahan salah guna kedudukan, pecah amanah jenayah dan pengubahan wang haram yang paling teruk.

Mohd Nazlan yang mensabitkan Najib terhadap kesemua tujuh tuduhan dan memerintahkan bekas Perdana Menteri itu dipenjara 12 tahun, dan didenda RM210 juta, berkata ia bukan sahaja disebabkan oleh alasan bagaimana jenayah itu dilakukan, malah kes berprofil tinggi berkenaan turut membabitkan sejumlah wang yang sangat besar.

Melalui alasan penghakiman penuh setebal 801 muka surat itu, Mohd Nazlan, berkata kes terbabit mempunyai elemen yang memberikan kesan ke atas kepentingan awam kerana dana RM42 juta itu adalah milik Kementerian Kewangan (Diperbadankan) (MKD) yang berkemungkinan berasal daripada dana pencen Kumpulan Wang Persaraan (Diperbadankan) (KWAP) berjumlah RM4 bilion.

"Dan yang paling penting ia membabitkan individu yang pada ketika itu berada dalam pada tertinggi dalam kerajaan," katanya.

Pada 28 Julai lalu, Mohd Nazlan memerintahkan Najib dipenjarakan 10 tahun masing-masing bagi tiga tuduhan pecah amanah wang RM42 juta milik SRC.

Hakim turut memerintahkan Najib dipenjara 12 tahun dan denda RM210 juta (jika gagal bayar, lima tahun penjara) bagi tuduhan menyalahgunakan kedudukan.

Bagi tuduhan pengubahan wang haram pula, Mohd Nazlan memerintahkan Najib dipenjara 10 tahun bagi setiap tuduhan.

Sementara itu, Mohd Nazlan berkata, Najib selaku tertuduh tidak menunjukkan penyesalan, malah mempertahankan pembelaan beliau tidak mengetahui mengenai wang RM42 juta milik SRC itu dalam rayuannya bagi diringankan hukuman.

"Tetapi saya tidak boleh menafikan beliau adalah Perdana Menteri negara ini dan tidak boleh mempersoalkan sumbangannya untuk kebaikan dan kesejahteraan masyarakat dalam pelbagai cara kerana beliau adalah Perdana Menteri selama sembilan tahun.

"Sejarah politik akan terus diperdebatkan sama ada dari segi keseimbangan, beliau melakukan lebih banyak kebaikan daripada keburukan.

"Walau apa pun, ia adalah tidak selari dengan idea sesebuah pentadbiran negara yang bersih daripada rasuah yang tidak boleh bertolak ansur dengan sebarang penyalahgunaan kuasa," katanya.

Mahkamah Rayuan menetapkan pada 15 Oktober ini bagi pengurusan kes rayuan Najib terhadap sabitan dan hukuman terhadapnya.
"""

In [16]:
string2 = """
Gabungan parti Warisan, Pakatan Harapan, dan Upko hari ini mendedahkan calon-calon masing-masing untuk pilihan raya negeri Sabah, tetapi ketika pengumuman itu berlangsung, perwakilan PKR di dewan itu dilihat ‘gelisah’ seperti ‘tidak senang duduk’.

Sekumpulan anggota PKR kemudian dilihat meninggalkan dewan di Pusat Konvensyen Antarabangsa Sabah di Kota Kinabalu selepas berbincang dengan ketua PKR Sabah Christina Liew.

Semakan senarai-senarai calon berkenaan mendapati PKR hanya memperolehi separuh daripada jumlah kerusi yang diharapkan.

Semalam, PKR Sabah mengumumkan akan bertanding di 14 kerusi tetapi ketika Presiden Warisan Shafie Apdal mengumumkan calon gabungan tersebut hari ini, PKR hanya diberikan tujuh kerusi untuk bertanding.

Kerusi yang diberikan adalah Api-Api, Inanam, Tempasuk, Tamparuli, Matunggong, Klias, dan Sook.

Klias dan Sook adalah dua kerusi yang diberikan kepada PKR, sementara lima kerusi selebihnya pernah ditandingi oleh PKR pada pilihan raya umum 2018.

Dalam pengumuman PKR Sabah semalam, parti itu menjangkakan Warisan akan turut menyerahkan kerusi Kemabong, Membakut, dan Petagas kepada mereka.

Walau bagaimanapun, Warisan menyerahkan kerusi Kemabong kepada Upko dan mengekalkan bertanding untuk kerusi Membakut dan Petagas.

PKR juga menuntut empat daripada 13 kerusi baru yang diperkenalkan iaitu Segama, Limbahau, Sungai Manila, dan Pintasan tetapi Warisan membolot semua kerusi itu.

Sebagai pertukaran untuk kerusi yang diintainya, PKR bersedia untuk menyerahkan kerusi Kadaimaian, Kuala Penyu, dan Karanaan. Namun, ini dijangka tidak akan berlaku memandangkan parti tersebut tidak berpuas hati dengan agihan kerusi seperti yang diharapkan itu.

Selepas perwakilan dari PKR dan Liew keluar dari dewan tersebut, wartawan kemudian menyusuri Liew untuk mendapatkan penjelasannya.

Walau bagaimanapun, Liew enggan memberikan sebarang komen dan berkata bahawa dia ingin ke tandas.

Liew dan perwakilan PKR kemudian tidak kembali ke dalam dewan tersebut.

Apabila calon pilihan raya yang diumumkan diminta naik ke atas pentas untuk sesi bergambar, Liew tidak kelihatan.

Bilangan kerusi yang ditandingi oleh PKR kali ini hanya kurang satu kerusi daripada yang ditandingi parti itu pada PRU 2018.

Dalam perkembangan berkaitan, DAP dan Amanah dikatakan tidak mempunyai sebarang masalah dengan kerusi yang diberikan untuk PRN Sabah.

Sementara itu, Presiden Upko Madius Tangau enggan mengulas adakah dia berpuas hati dengan agihan kerusi tersebut. Madius kekal di majlis tersebut sehingga ia berakhir.

Partinya diberikan 12 kerusi, iaitu lebih tujuh kerusi berbanding PRU lalu.

DAP dan Amanah akan bertanding di bawah logo Warisan sementara PKR dan Upko akan menggunakan logo masing-masing.

DAP akan bertanding di tujuh kerusi, jumlah yang sama seperti yang mereka tandingi pada PRU lalu, sementara Amanah diberi satu kerusi.

Warisan akan bertanding sebanyak 54 kerusi.

Perkembangan terbaru ini mungkin mencetuskan pergeseran di antara PKR dan Warisan. PKR boleh memilih untuk bertanding di lebih banyak kerusi daripada 14 yang dituntutnya manakala Warisan juga boleh bertanding di kerusi sekutunya.

Barisan pemimpin tertinggi PKR dan Warisan hanya mempunyai dua hari sebelum hari penamaan calon pada Sabtu untuk mengurangkan pergeseran.
"""

In [17]:
string3 = """
Penubuhan universiti sukan seperti diutarakan Ketua Unit Sukan Kementerian Pengajian Tinggi, Dr Pekan Ramli dan disokong Pakar Pembangunan Sukan dan Reakreasi Luar, Universiti Pendidikan Sultan Idris (UPSI), Prof Dr Md Amin Md Taaf seperti disiarkan akhbar ini, memberikan sinar harapan kepada kewujudan institusi sedemikian.

Ia menjadi impian atlet negara untuk mengejar kejayaan dalam bidang sukan dan kecemerlangan dalam akademik untuk menjamin masa depan lebih baik apabila bersara daripada arena sukan kelak.

Pelbagai pandangan, idea, kaedah, bukti dan cadangan dilontarkan pakar berikutan pentingnya universiti sukan yang akan memberi impak besar sama ada pada peringkat kebangsaan mahupun antarabangsa.

Negara lain sudah lama meraih laba dengan kewujudan universiti sukan seperti China, Korea, Japan, Taiwan, India dan Vietnam. Mereka menghasilkan atlet universiti yang mempamerkan keputusan cemerlang pada peringkat tinggi seperti Sukan Olimpik, Kejohanan Dunia dan Sukan Asia.

Justeru, kejayaan mereka perlu dijadikan rujukan demi memajukan sukan tanah air. Jika kita merujuk pendekatan Asia, kewujudan universiti sukan penting dan memberi kesan positif dalam melonjakkan prestasi sukan lebih optimum.

Namun, jika kita melihat pendekatan Eropah, universiti sukan bukan antara organisasi atau institusi penting yang diberi perhatian dalam menyumbang kepada pemenang pingat.

Antara isu dalam universiti sukan ialah kos tinggi, lokasi, prasarana sukan, pertindihan kursus dengan universiti sedia ada dan impak terhadap dunia sukan negara hingga mengundang persoalan kewajaran dan kerelevanan penubuhannya.

Namun sebagai bekas atlet memanah negara dan Olympian (OLY) di Sukan Olimpik 2004 di Athens, Greece serta bekas pelajar Sekolah Sukan Bukit Jalil hingga berjaya dalam dunia akademik, saya mendapati terdapat beberapa faktor sering menjadi halangan dalam rutin harian mereka.

Antaranya, faktor masa yang terpaksa bergegas menghadiri kuliah selepas tamat sesi latihan yang mengambil masa 15 hingga 20 minit dengan menunggang motosikal; kereta (20-30 minit) atau pengangkutan disediakan Majlis Sukan Negara (MSN) ke Universiti Putra Malaysia (UPM).

Jika mereka menuntut di Universiti Teknologi MARA (UiTM) atau Universiti Malaya (UM), ia mungkin lebih lama.

Walaupun di universiti tersedia dengan kemudahan kolej dan kemudahan sukan, mereka memilih pulang ke MSN untuk menjalani latihan bersama pasukan dan jurulatih di padang atau gelanggang latihan rasmi.

Ini berlanjutan selagi bergelar atlet negara yang perlu memastikan prestasi sentiasa meningkat dari semasa ke semasa tanpa mengabaikan tugas sebagai pelajar.

Alangkah baiknya jika sebahagian Sekolah Sukan Bukit Jalil itu sendiri dijadikan Kolej atau Universiti Sukan Malaysia kerana lengkap dari segi kemudahan prasarana sukannya dan proses pengajaran dan pembelajaran (PdP) dalam bidang Sains Sukan, Kejurulatihan, Pendidikan Jasmani dan setaraf dengannya.

Pengambilan setiap semester pula hanya terhad kepada atlet berstatus kebangsaan dan antarabangsa sahaja supaya hasrat melahirkan lebih ramai atlet bertaraf Olimpik mudah direalisasikan.

Contohnya, bekas atlet lompat bergalah negara, Roslinda Samsu yang juga pemenang pingat perak Sukan Asia Doha 2006 dan Penerima Anugerah Khas Majlis Anugerah Sukan KPT 2012, terpaksa mengambil masa lebih kurang sembilan tahun untuk menamatkan ijazah Sarjana Muda Pendidikan Jasmani di UPM sepanjang 14 tahun terbabit dalam sukan olahraga.

Sepanjang tempoh bergelar atlet kebangsaan dan mahasiswa, beliau juga memenangi pingat Emas Sukan SEA empat siri berturut-turut pada 2005, 2007, 2009 dan 2011.

Begitu juga atlet kebangsaan seperti Leong Mun Yee (UPM); Pandalela Renong (UM); Bryan Nickson Lomas (UM); Cheng Chu Sian (UPM); Marbawi Sulaiman (UiTM) dan Norasheela Khalid (UPM).

Jika disenaraikan, mungkin lebih ramai lagi. Namun, pernah terlintas di fikiran mengapa hanya atlet dari sukan terjun yang dapat memenangi pingat di Sukan Olimpik? Bagaimana dengan atlet lain yang juga layak secara merit? Apakah kekangan atau masalah dihadapi sebagai atlet dan mahasiswa?

Adakah kewujudan universiti sukan akan memberi impak besar kepada kemajuan sukan negara? Jika dirancang dan diatur dengan cekap dan sistematik, ia perkara tidak mustahil dicapai.
"""

In [18]:
cleaning(string)

'KUALA LUMPUR: Hakim Mahkamah Tinggi, Mohd Nazlan Mohd Ghazali menyifatkan kes penyelewengan dana RM42 juta milik SRC International Sdn Bhd dihadapi Datuk Seri Najib Razak adalah kesalahan salah guna kedudukan, pecah amanah jenayah dan pengubahan wang haram yang paling teruk. Mohd Nazlan yang mensabitkan Najib terhadap kesemua tujuh tuduhan dan memerintahkan bekas Perdana Menteri itu dipenjara 12 tahun, dan didenda RM210 juta, berkata ia bukan sahaja disebabkan oleh alasan bagaimana jenayah itu dilakukan, malah kes berprofil tinggi berkenaan turut membabitkan sejumlah wang yang sangat besar. Melalui alasan penghakiman penuh setebal 801 muka surat itu, Mohd Nazlan, berkata kes terbabit mempunyai elemen yang memberikan kesan ke atas kepentingan awam kerana dana RM42 juta itu adalah milik Kementerian Kewangan (Diperbadankan) (MKD) yang berkemungkinan berasal daripada dana pencen Kumpulan Wang Persaraan (Diperbadankan) (KWAP) berjumlah RM4 bilion. "Dan yang paling penting ia membabitkan in

In [19]:
# splitted_string = malaya.text.function.split_into_sentences(transformer_textcleaning(string))
# splitted_string2 = malaya.text.function.split_into_sentences(transformer_textcleaning(string2))

In [20]:
encoded = encoder.encode(f'tajuk: {transformer_textcleaning(string)}') + [1]
f, b, n = sess.run([model.fast_result, model.beam_result, model.nucleus_result], 
             feed_dict = {model.X: [encoded], model.top_p: 0.7})

(encoder.decode(f[0].tolist()), 
encoder.decode(b[0].tolist()), 
encoder.decode(n[0].tolist()))

('Kes Najib salah guna dominasi Najib, undang wang haram',
 'Kes SRC Najib salah guna dominasi Najib',
 'Kes pecah amanah Najib salah guna dominasi pegawai negeri, kata hakim')

In [21]:
encoded = encoder.encode(f'ringkasan: {transformer_textcleaning(string)}') + [1]
f, b, n = sess.run([model.fast_result, model.beam_result, model.nucleus_result], 
             feed_dict = {model.X: [encoded], model.top_p: 0.7})

(encoder.decode(f[0].tolist()), 
encoder.decode(b[0].tolist()), 
encoder.decode(n[0].tolist()))

('Kes SRC Najib salah guna dominasi Najib, pecah amanah jenayah',
 'Kes SRC Najib salah guna dominasi Najib, pecah amanah jenayah',
 'Kes Najib salah guna dominasi isytihar, kalah, kekal penjara')

In [22]:
encoded = encoder.encode(f'tajuk: {transformer_textcleaning(string2)}') + [1]
f, b, n = sess.run([model.fast_result, model.beam_result, model.nucleus_result], 
             feed_dict = {model.X: [encoded], model.top_p: 0.7})

(encoder.decode(f[0].tolist()), 
encoder.decode(b[0].tolist()), 
encoder.decode(n[0].tolist()))

('Dua calon PKR Sabah umum calon',
 'Dua calon PKR Sabah umum calon',
 '200,000 calon PKR Sabah pada 2018')

In [23]:
encoded = encoder.encode(f'tajuk: {transformer_textcleaning(string3)}') + [1]
f, b, n = sess.run([model.fast_result, model.beam_result, model.nucleus_result], 
             feed_dict = {model.X: [encoded], model.top_p: 0.7})

(encoder.decode(f[0].tolist()), 
encoder.decode(b[0].tolist()), 
encoder.decode(n[0].tolist()))

('Kejayaan sukan lebih baik',
 'Kejayaan sukan lebih baik',
 'Kejayaan sukan seiringan bukan jaminan Sukan')

In [24]:
encoded = encoder.encode(f'ringkasan: {transformer_textcleaning(string3)}') + [1]
f, b, n = sess.run([model.fast_result, model.beam_result, model.nucleus_result], 
             feed_dict = {model.X: [encoded], model.top_p: 0.7})

(encoder.decode(f[0].tolist()), 
encoder.decode(b[0].tolist()), 
encoder.decode(n[0].tolist()))

('Kejayaan sukan lebih baik',
 'Kejayaan sukan lebih baik',
 'Tigers dan PIL 2018; Ibu bapa cuai Kinninja Kal')

In [25]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, 'transformer-small/model.ckpt')

'transformer-small/model.ckpt'

In [26]:
strings = ','.join(
    [
        n.name
        for n in tf.get_default_graph().as_graph_def().node
        if ('Variable' in n.op
        or 'Placeholder' in n.name
        or 'greedy' in n.name
        or 'beam' in n.name
        or 'nucleus' in n.name
        or 'alphas' in n.name
        or 'self/Softmax' in n.name)
        and 'adam' not in n.name
        and 'beta' not in n.name
        and 'global_step' not in n.name
        and 'modality' not in n.name
        and 'Assign' not in n.name
    ]
)
strings.split(',')

['Placeholder',
 'Placeholder_1',
 'Placeholder_2',
 'transformer/body/target_space_embedding/kernel/Read/ReadVariableOp',
 'transformer/parallel_0_4/transformer/transformer/body/target_space_embedding/ReadVariableOp',
 'transformer/body/encoder/layer_0/self_attention/layer_prepostprocess/layer_norm/layer_norm_scale/Read/ReadVariableOp',
 'transformer/body/encoder/layer_0/self_attention/layer_prepostprocess/layer_norm/layer_norm_bias/Read/ReadVariableOp',
 'transformer/parallel_0_4/transformer/transformer/body/encoder/layer_0/self_attention/layer_prepostprocess/layer_norm/ReadVariableOp',
 'transformer/parallel_0_4/transformer/transformer/body/encoder/layer_0/self_attention/layer_prepostprocess/layer_norm/ReadVariableOp_1',
 'transformer/body/encoder/layer_0/self_attention/multihead_attention/q/kernel/Read/ReadVariableOp',
 'transformer/parallel_0_4/transformer/transformer/body/encoder/layer_0/self_attention/multihead_attention/q/Tensordot/ReadVariableOp',
 'transformer/body/encoder/la

In [27]:
def freeze_graph(model_dir, output_node_names):

    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            'directory: %s' % model_dir
        )

    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path

    absolute_model_dir = '/'.join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + '/frozen_model.pb'
    clear_devices = True
    with tf.Session(graph = tf.Graph()) as sess:
        saver = tf.train.import_meta_graph(
            input_checkpoint + '.meta', clear_devices = clear_devices
        )
        saver.restore(sess, input_checkpoint)
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess,
            tf.get_default_graph().as_graph_def(),
            output_node_names.split(','),
        )
        with tf.gfile.GFile(output_graph, 'wb') as f:
            f.write(output_graph_def.SerializeToString())
        print('%d ops in the final graph.' % len(output_graph_def.node))

In [28]:
freeze_graph('transformer-small', strings)

INFO:tensorflow:Restoring parameters from transformer-small/model.ckpt


INFO:tensorflow:Restoring parameters from transformer-small/model.ckpt


Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`


Instructions for updating:
Use `tf.compat.v1.graph_util.convert_variables_to_constants`


Instructions for updating:
Use `tf.compat.v1.graph_util.extract_sub_graph`


...

INFO:tensorflow:Froze 188 variables.


INFO:tensorflow:Froze 188 variables.


INFO:tensorflow:Converted 188 variables to const ops.


INFO:tensorflow:Converted 188 variables to const ops.


15839 ops in the final graph.


In [29]:
def load_graph(frozen_graph_filename):
    with tf.gfile.GFile(frozen_graph_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)
    return graph

In [30]:
g = load_graph('transformer-small/frozen_model.pb')
x = g.get_tensor_by_name('import/Placeholder:0')
greedy = g.get_tensor_by_name('import/greedy:0')
beam = g.get_tensor_by_name('import/beam:0')
nucleus = g.get_tensor_by_name('import/nucleus:0')
top_p = g.get_tensor_by_name('import/Placeholder_2:0')
test_sess = tf.InteractiveSession(graph = g)

In [31]:
g, b, n = test_sess.run([greedy, beam, nucleus], feed_dict = {x:[encoded],
                                                             top_p: 0.7})

In [32]:
(encoder.decode(f[0].tolist()), 
encoder.decode(b[0].tolist()), 
encoder.decode(n[0].tolist()))

('Kejayaan sukan lebih baik',
 'Kejayaan sukan lebih baik',
 'Kejayaan sukan besar bukan jaminan Olimpik merit - KBS')